In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

warnings.filterwarnings("ignore")

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

train = pd.read_csv("train_수정_2.csv")
test = pd.read_csv("train_수정_2.csv")
loan_result = pd.read_csv("loan_result.csv", usecols=['application_id', 'loan_rate', 'loan_limit', 'product_id'])

In [3]:
train = pd.merge(train, loan_result, how='inner', on='application_id')
test = pd.merge(test, loan_result, how='inner', on='application_id')

MemoryError: Unable to allocate 62.0 GiB for an array with shape (24, 346499593) and data type float64

### 6-1. loan_limit / desired_amount

In [ ]:
train['loan_limit / desired_amount'] = train['loan_limit'] /train['desired_amount']
test['loan_limit / desired_amount'] = test['loan_limit'] /test['desired_amount']

> 이상치 처리

In [ ]:
train.loc[train['desired_amount']==0, 'loan_limit / desired_amount']=0
test.loc[test['desired_amount']==0, 'loan_limit / desired_amount']=0

### 6-2. loan_rate_min / loan_rate

In [ ]:
train['loan_rate_min / loan_rate'] = train['loan_rate'].min() / train['loan_rate']
test['loan_rate_min / loan_rate'] = test['loan_rate'].min() / test['loan_rate']

### 6-3. existing_loan_rate

In [ ]:
train['existing_loan_rate'] = (train['existing_loan_amt']/train['yearly_income'])*100
train['existing_loan_rate'] = train['existing_loan_rate'].fillna(0)
train = train.replace({'existing_loan_rate' : inf}, 0)

In [ ]:
test['existing_loan_rate'] = (test['existing_loan_amt']/test['yearly_income'])*100
test['existing_loan_rate'] = test['existing_loan_rate'].fillna(0)
test = test.replace({'existing_loan_rate' : inf}, 0)

## 7 . 훈련 데이터셋 리샘플링

> 데이터 타겟 값의 불균형을 보완하기 위하여 데이터 리샘플링 진행
> 다양한 리샘플링 모델 중 복합 샘플링에 속하는 SMOTEENN 모듈 활용

In [4]:
from imblearn.combine import *

In [ ]:
train_X, train_y = train.drop('is_applied', axis=1), train['is_applied']

In [ ]:
undersample = SMOTEENN(random_state=0)
train_X, train_y = undersample.fit_resample(train_X, train_y)